In [13]:
from scipy.stats import f, norm, t, chi
from math import sqrt
from copy import deepcopy
import matplotlib.pyplot as plt
from numpy import random

def T(X): # функция транспонирование матрицы X
    X_T = []
    for j in range(len(X[0])):
        X_T.append([])
        for i in range(len(X)):
            X_T[j].append(X[i][j])
    return X_T

def multiply(X, Y): #умножение двух матриц X и Y
    result = []
    for i in range(len(X)):
        result.append([])
        for j in range(len(Y[0])):
            result[i].append(sum([X[i][k] * Y[k][j] for k in range(len(X[i]))]))
    return result

def minor(A, row, column):#создает минор матрицы A, удаляя строку row и столбец column из исходной матрицы
    M = []
    for i in range(len(A)):
        if i != row:
            M.append([])
            for j in range(len(A[i])):
                if j != column:
                    M[i if i < row else i - 1].append(A[i][j])
    return M

def det(A):# вычисляет определитель матрицы
    if len(A) == 1:
        return A[0][0]
    if len(A) == 2:
        return A[0][0] * A[1][1] - A[0][1]* A[1][0]
    return sum([(-1)**i * A[0][i] * det(minor(A, 0, i)) for i in range(len(A[0]))])

def reverse(A):#вычисляет обратную матрицу для матрицы A
    det_A = det(A)
    assert det_A != 0, "bad reverse"
    result = []
    for i in range(len(A)):
        result.append([])
        for j in range(len(A[0])):
            result[i].append((-1)**(i + j) * det(minor(A, i, j)) / det_A)
    return T(result)


funcs = [lambda x: 1, lambda x: 1/x, lambda x: 1/(5-x)]
betta_is = [5, 6, 7]
sigma = 2
alpha = 0.06

x_left = 1
x_right = 4
n =25
h = (x_right - x_left) / (n - 1)

x = [x_left + i * h for i in range(n)]#значения аргументов функций
y = [[sum([betta_is[k] * funcs[k](x[i]) for k in range(len(funcs))])]#значения функций
     for i in range(len(x))]
for i in range(len(y)):
    y[i][0] += random.normal(0, sigma)#добавляет случайный шум к результатам, вычисленным в предыдущем шаге.


X = []
for i in range(len(x)):
    X.append([])
    for k in range(len(funcs)):
        X[i].append(funcs[k](x[i]))#для каждого элемента x[i] в списке x, вычисляется значение функции funcs[k] и добавляем его в соответствующий внутренний список X[i].

A = multiply(T(X), X)#умножения транспонированной матрицы T(X) на исходную матрицу X

reverse_A = reverse(A)

#оценка коэффициентов модели, полученная после умножение ( умножение обратной матрицы на транспонированную матрицу) на матрицу y. .
betta_estimation = multiply(multiply(reverse_A, T(X)), y)#оценка коэффициентов модели
d_betta_without_sigma = reverse_A #оценки дисперсии оценок коэффициентов модели.

u = norm.ppf(1 - alpha / 2, loc=0, scale=1) #квантиль стандартного нормального распределения
t_q = t.ppf(1 - alpha / 2, n - len(betta_is))#квантиль распределения Стьюдента.
f_q = f.ppf(1 - alpha, len(betta_is), n - len(betta_is))#квантиль распределения Фишера

print("Оценки:")
for i in range(len(betta_estimation)):
    print(f"betta{i + 1} = {betta_estimation[i][0]}, истинное: {betta_is[i]}") #сопоставляет оценки betta_estimationс с  bettas "истинными" значениями
print()

y_estimation = multiply(X, betta_estimation)# вычисления оценок y_estimation.

#оценку стандартной ошибки регрессии (статистическую меру разброса остатков)
#корень среднего квадрата остатков регрессии, где остаток - это разница между фактическим и предсказанным значением зависимой переменной
s = sqrt(1 / (n - len(betta_is)) * sum([(y[i][0] - y_estimation[i][0])**2 for i in range(n)]))

#ПОСТРОЕНИЕ ДОВ ИНТЕРВАЛОВ
borders_with_known = []
for i in range(len(betta_estimation)):
    borders_with_known.append([])
    borders_with_known[i].append(betta_estimation[i][0] - u * sigma * sqrt(d_betta_without_sigma[i][i])) #нижняя граница интервала
    borders_with_known[i].append(betta_estimation[i][0] + u * sigma * sqrt(d_betta_without_sigma[i][i])) #верхняя граница интервала

borders_with_unknown = []
for i in range(len(betta_estimation)):
    borders_with_unknown.append([]) #нижняя граница оценки бетта -квантиль Стьюдента*оценка стандратной ошибки*корень из оценок дисперсии
    borders_with_unknown[i].append(betta_estimation[i][0] - t_q * s * sqrt(d_betta_without_sigma[i][i])) #нижняя
    borders_with_unknown[i].append(betta_estimation[i][0] + t_q * s * sqrt(d_betta_without_sigma[i][i])) #верхняя

print("При известном sigma:")
for i in range(len(betta_estimation)):
    print(f"{borders_with_known[i][0]} <= betta{i + 1} <= {borders_with_known[i][1]}")
print()

print("При неизвестном sigma:")
for i in range(len(betta_estimation)):
    print(f"{borders_with_unknown[i][0]} <= betta{i + 1} <= {borders_with_unknown[i][1]}")
print()

print("Проверка гипотез: H0: betta_i == 0, H1: betta_i != 0") #равен ли параметр betta[i] нулю
for i in range(len(betta_estimation)):
    if borders_with_known[i][0] <= 0 <= borders_with_known[i][1]:
        print(f'betta{i+1}: H0 при известном, H0 при неизвестном оценка незначима')
    else:
        print(f'betta{i+1}: H1 при известном, H1 при неизвестном оценка значима')
print()




print("Проверка гипотез: H0: betta_i истинная") # равен ли betta[i] своему истинному значению
for i in range(len(betta_estimation)):
    print(f'betta{i+1}: {"H0" if borders_with_known[i][0] <= betta_is[i] <= borders_with_known[i][1] else "H1"} \
при известном, {"H0" if borders_with_unknown[i][0] <= betta_is[i] <= borders_with_unknown[i][1] else "H1"} при неизвестном')
print() #Выводится "H0", если истинное бетта входит в доверительный интервал, и "H1" в противном случае.

#5 ЛАБА

def Q(bettas_exp):#вектор экспериментальных или альтернативных значений параметров betta
    betta_diff = [[betta_estimation[k][0] - bettas_exp[k]] for k in range(len(bettas_exp))]#разницу оценки и экспериментальой бетта
    return multiply(multiply(T(betta_diff), A), betta_diff)[0][0]#одно значения из матрицы=умножение (умножение транспонированной матрицы разницы betta_estimation и bettas_exp на матрицу A)на разницу betta_estimation и bettas_exp

#проверку гипотез о параметрах betta в двух случаях: когда  sigma известно и когда оно неизвестно.
for exp in [[0, 0, 0], betta_is]:#2 итерации, в первой итерации переменная exp принимает значение [0, 0, 0], а во второй итерации - значения из списка bettas.
    print(f'Проверка гипотезы: H0: betta == {exp}')
    print("При неизвестной:") #cумма квадратов ошибок (SSE)=len(exp) * s**2
    if Q(exp)/(len(exp) * s**2) < f_q:#Если значение критерия меньше квантиля Фишера
        print(f'{Q(exp)/(len(exp) * s**2)} < {f_q} => H0')#то нулевая гипотеза H0 принимается -оценка незначима для 0 0 0
    else:
        print(f'{Q(exp)/(len(exp) * s**2)} > {f_q} => H1')#иначе принимается альтернативная гипотеза H1.
    print("При известной:")
    if Q(exp)/(sigma**2) < chi.ppf(1 - alpha / 2, len(exp)): #Если значение критерия меньше квантиля хи-квадратом
        print(f'{Q(exp)/(sigma**2)} < {chi.ppf(1 - alpha, len(exp))} => H0')#то нулевая гипотеза H0 принимается
    else:
        print(f'{Q(exp)/(sigma**2)} > {chi.ppf(1 - alpha, len(exp))} => H1')##иначе принимается альтернативная гипотеза H1.
    print()





Оценки:
betta1 = 5.262920206807169, истинное: 5
betta2 = 7.627797310383841, истинное: 6
betta3 = 5.792932959601014, истинное: 7

При известном sigma:
0.15107356396848992 <= betta1 <= 10.374766849645848
1.8957794994123374 <= betta2 <= 13.359815121355343
0.06091514862952074 <= betta3 <= 11.524950770572508

При неизвестном sigma:
-0.016980545642803158 <= betta1 <= 10.542820959257142
1.7073370016608864 <= betta2 <= 13.548257619106796
-0.1275273491219302 <= betta3 <= 11.713393268323959

Проверка гипотез: H0: betta_i == 0, H1: betta_i != 0
betta1: H1 при известном, H1 при неизвестном оценка значима
betta2: H1 при известном, H1 при неизвестном оценка значима
betta3: H1 при известном, H1 при неизвестном оценка значима

Проверка гипотез: H0: betta_i истинная
betta1: H0 при известном, H0 при неизвестном
betta2: H0 при известном, H0 при неизвестном
betta3: H0 при известном, H0 при неизвестном

Проверка гипотезы: H0: betta == [0, 0, 0]
При неизвестной:
292.11036817419705 > 2.862314711555532 => H1


6 лаба

In [14]:
%autosave 60

Autosaving every 60 seconds


In [15]:
from math import log

def create_intervals_starts(data):#интервалы для группировки данных
    min_value = min(data) #минимальное значение в массиве.
    max_value = max(data) #максимальное значение в массиве.
    data_count = len(data) #количество элементов в массиве.
    intervals_count = int(log(data_count, 2)) + 1 #количество интервалов,  берется целая часть от значения логарифма по осн 2 +добавляется единица, чтобы гарантировать, что будет как минимум один интервал
    interval_length = (max_value - min_value) / intervals_count #длину каждого интервала.
    intervals = [min_value + i * interval_length for i in range(intervals_count)] # список интервалов.
    i = 0
    while i + 1 < len(intervals):#проверку, чтобы объединить соседние интервалы, если они не содержат значений из массива.
        if sum(intervals[i] <= value < intervals[i + 1] for value in data) == 0:#Если сумма значений из интервала равна 0, это означает, что в заданном интервале нет ни одного значения из данных
            intervals.pop(i + 1)#удаляем верхнюю границу этого интервала, объединяя его с предыдущим.
        else:
            i += 1#Если хотя бы одно значение из массива data попало в текущий интервал, то мы переходим к следующему интервалу.
    intervals.append(max(data) + 1)#добавляем верхнюю границу для последнего интервала.
    return intervals

print("Проверка гипотезы H0: x и y независимые")
x_intervals = create_intervals_starts(x) #разделить значения переменной x на интервалы и создать список границ этих интервалов.
y_intervals = create_intervals_starts([p[0] for p in y])#список границ интервалов для переменной y- создать новый список, извлекая первый элемент из каждого элемента списка y

result_table = [] #таблицу результатов.
for i in range(len(x_intervals) - 1):#для каждого интервала и исключить последний интервал
    result_table.append([])#создается вложенный пустой список в result_table, представляющий строку в таблице.
    for j in range(len(y_intervals) - 1):#цикл выполняется для каждого интервала-список интервалов значений для переменной y.
        result_table[i].append(sum(x_intervals[i] <= x_value < x_intervals[i + 1] and y_intervals[j] <= y_value[0] < y_intervals[j + 1] for x_value, y_value in zip(x, y)))
for line in result_table:#добавляет в текущую строку result_table сумму значений, которые попадают в текущие интервалы для переменных x и y,
    print(line)


h = 0 # тест на независимость переменных x и y на основе таблицы сопряженности
x_sum = sum(result_table[i]) #i - это индекс текущей строки, по которой мы проходимся во внешнем цикле for i in range(len(result_table)).
eta_sum = sum(result_table[i][j] for i in range(len(result_table)))#по каждой строке i таблицы result_table и берем значение из столбца j для каждой строки. Затем мы суммируем эти значения..
for i in range(len(result_table)):#проходит по каждой строке в таблице.
    for j in range(len(result_table[0])):#по индексам столбцов в первой строке таблицы-ищем длину первой строки, чтобы определить количество столбцов в таблице
        h += (result_table[i][j])**2 / (x_sum  * eta_sum)#каждый элемент таблицы возводится в квадрат и делится на произведение сумм значений по строкам и столбцам.
h *= n*(h-1) #Значение h умножается на размер выборки n


chi_q = chi.ppf(1 - alpha, (len(result_table) - 1) * (len(result_table[0]) - 1))**2 #функция вычисляет квантиль хи-квадрат распределения для заданного уровня значимости alpha и количества степеней свободы, которое вычисляется как произведение количества строк минус один и количества столбцов минус один.
if h < chi_q:
    print(f'{h} < {chi_q} => принимается H0 => независимы')
else:
    print(f'{h} > {chi_q} => принимается H1 => зависимы')

# Значение статистики хи-квадрат
print("\nЗначение статистики хи-квадрат chi2:")
print(h)

# Критическое значение хи-квадрат
print("\nКритическое значение хи-квадрат chi2_quantile:")
print(chi_q)


h = 0 # тест на независимость переменных x и y на основе таблицы сопряженности

# Сумма по строкам
#x_sum = [sum(row) for row in result_table]
#print("Сумма по строкам:")
#for i in range(len(x_sum)):
    #print(f"Строка {i + 1}: {x_sum[i]}")

# Сумма по столбцам с учетом строк
#print("\nСумма по столбцам с учетом строк:")
#for j in range(len(result_table[0])):
    #eta_sum = sum(result_table[i][j] for i in range(len(result_table)))
    #print(f"Столбец {j + 1}: {eta_sum}")


Проверка гипотезы H0: x и y независимые
[1, 0, 0, 1, 3]
[1, 1, 2, 1, 0]
[1, 0, 2, 2, 0]
[2, 0, 2, 1, 0]
[0, 0, 3, 1, 1]
79.31249999999997 > 25.594994767626055 => принимается H1 => зависимы

Значение статистики хи-квадрат chi2:
79.31249999999997

Критическое значение хи-квадрат chi2_quantile:
25.594994767626055
